In [1]:
import pandas as pd
import numpy as np
import nltk
# nltk.download('punkt')
import re
from bs4 import BeautifulSoup
from sklearn.decomposition import TruncatedSVD
from keras.models import Sequential
from keras.layers import Dense, Embedding, LSTM, SpatialDropout1D
from keras.callbacks import EarlyStopping
from keras.utils import to_categorical
import tensorflow as tf
from sklearn.preprocessing import LabelEncoder
from sklearn.svm import LinearSVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report
from sklearn.metrics import accuracy_score

In [2]:
df = pd.read_csv('Twitter_Data.csv')
df.head()

,clean_text,category
0,when modi promised “minimum government maximum...,-1.0
1,talk all the nonsense and continue all the dra...,0.0
2,what did just say vote for modi welcome bjp t...,1.0
3,asking his supporters prefix chowkidar their n...,1.0
4,answer who among these the most powerful world...,1.0


In [3]:
df=df.dropna()
df['category'] = df['category'].astype(int)
df = df.reset_index(drop=True)
df['category'].value_counts()

 1    72249
 0    55211
-1    35509
Name: category, dtype: int64

In [4]:
# df=df[df['category']!=0]
df['category'].value_counts()

 1    72249
 0    55211
-1    35509
Name: category, dtype: int64

In [5]:
df = df.sample(frac=1).reset_index(drop=True) #shuffling of tweets
data=df[df['category']==-1][:35000]
data=data.append(df[df['category']==0][:35000])
data=data.append(df[df['category']==1][:35000])
data = data.reset_index(drop=True)
display(data['category'].value_counts())
data

C:\Users\CPORGALINAS\AppData\Local\Temp\ipykernel_5496\1387895886.py:3: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  data=data.append(df[df['category']==0][:35000])
C:\Users\CPORGALINAS\AppData\Local\Temp\ipykernel_5496\1387895886.py:4: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  data=data.append(df[df['category']==1][:35000])


-1    35000
 0    35000
 1    35000
Name: category, dtype: int64

,clean_text,category
0,soon international community gonna ask questio...,-1
1,lolz aik number chotia hai bharat wasiyo ther...,-1
2,image found under chowkidar modi todays times ...,-1
3,when modi stops water pakistan land then you a...,-1
4,modi take revenge for being jailed during emer...,-1
...,...,...
104995,\nvote for new bharat says modi telangana enli...,1
104996,modi journey common man\nlooks amazing\n\n,1
104997,not blind supporter any party saw and feel the...,1
104998,report nirav modi will remain the wandsworth p...,1


In [6]:
data['pre_process'] = data['clean_text'].apply(lambda x: " ".join(x.lower() for x in str(x).split()))
data['pre_process']=data['pre_process'].apply(lambda x: BeautifulSoup(x).get_text())
data['pre_process']=data['pre_process'].apply(lambda x: re.sub(r"http\S+", "", x))

C:\Users\CPORGALINAS\Documents\Machine Learning\Revalida 2\env\lib\site-packages\bs4\__init__.py:435: MarkupResemblesLocatorWarning: The input looks more like a filename than markup. You may want to open this file and pass the filehandle into Beautiful Soup.
  warnings.warn(


In [7]:
def contractions(s):
    s = re.sub(r"won't", "will not",s)
    s = re.sub(r"would't", "would not",s)
    s = re.sub(r"could't", "could not",s)
    s = re.sub(r"\'d", " would",s)
    s = re.sub(r"can\'t", "can not",s)
    s = re.sub(r"n\'t", " not", s)
    s= re.sub(r"\'re", " are", s)
    s = re.sub(r"\'s", " is", s)
    s = re.sub(r"\'ll", " will", s)
    s = re.sub(r"\'t", " not", s)
    s = re.sub(r"\'ve", " have", s)
    s = re.sub(r"\'m", " am", s)
    return s
data['pre_process']=data['pre_process'].apply(lambda x:contractions(x))

In [8]:
data['pre_process']=data['pre_process'].apply(lambda x: " ".join([re.sub('[^A-Za-z]+','', x) for x in nltk.word_tokenize(x)]))

In [9]:
from nltk.corpus import stopwords
stop = stopwords.words('english')
data['pre_process']=data['pre_process'].apply(lambda x: " ".join([x for x in x.split() if x not in stop]))

In [10]:
from nltk.stem import WordNetLemmatizer
lemmatizer = WordNetLemmatizer()
data['pre_process']=data['pre_process'].apply(lambda x: " ".join([lemmatizer.lemmatize(w) for w in nltk.word_tokenize(x)]))

In [11]:
data.head()

,clean_text,category,pre_process
0,soon international community gonna ask questio...,-1,soon international community gon na ask questi...
1,lolz aik number chotia hai bharat wasiyo ther...,-1,lolz aik number chotia hai bharat wasiyo dead ...
2,image found under chowkidar modi todays times ...,-1,image found chowkidar modi today time vision m...
3,when modi stops water pakistan land then you a...,-1,modi stop water pakistan land people would def...
4,modi take revenge for being jailed during emer...,-1,modi take revenge jailed emergency big announc...


In [12]:
from sklearn.model_selection import train_test_split
X_train,X_test,Y_train, Y_test = train_test_split(data['pre_process'], data['category'], test_size=0.25, random_state=30)
print("Train: ",X_train.shape,Y_train.shape,"Test: ",(X_test.shape,Y_test.shape))

Train:  (78750,) (78750,) Test:  ((26250,), (26250,))


In [13]:
print("TFIDF Vectorizer……")
from sklearn.feature_extraction.text import TfidfVectorizer
vectorizer= TfidfVectorizer()
X_train = vectorizer.fit_transform(X_train)
X_test = vectorizer.transform(X_test)

TFIDF Vectorizer……


In [14]:
# Training a Linear SVC
clf = LinearSVC(random_state=0)
clf.fit(X_train,Y_train)

LinearSVC(random_state=0)

In [15]:
# Predict on the test set
y_pred=clf.predict(X_test)

In [16]:
# Print accuracy score
print("Accuracy: ", accuracy_score(Y_test, y_pred))

Accuracy:  0.8788190476190476


In [17]:
print(classification_report(Y_test, y_pred))

              precision    recall  f1-score   support

          -1       0.87      0.86      0.87      8718
           0       0.87      0.94      0.90      8772
           1       0.89      0.84      0.87      8760

    accuracy                           0.88     26250
   macro avg       0.88      0.88      0.88     26250
weighted avg       0.88      0.88      0.88     26250



In [18]:
# # Train a Random Forest Classifier
clf = RandomForestClassifier(n_estimators=100)
clf.fit(X_train, Y_train)

RandomForestClassifier()

In [19]:
# # Predict on the test set
y_pred=clf.predict(X_test)

In [20]:
# Print accuracy score
print("Accuracy: ", accuracy_score(Y_test, y_pred))

Accuracy:  0.8357333333333333


In [21]:
print(classification_report(Y_test, y_pred))

              precision    recall  f1-score   support

          -1       0.84      0.79      0.81      8718
           0       0.81      0.93      0.87      8772
           1       0.86      0.79      0.82      8760

    accuracy                           0.84     26250
   macro avg       0.84      0.84      0.83     26250
weighted avg       0.84      0.84      0.83     26250



In [22]:
# Training for Logistic Regression
clf = LogisticRegression(max_iter=1000,solver='saga')
clf.fit(X_train, Y_train)

LogisticRegression(max_iter=1000, solver='saga')

In [23]:
# Predict on the test set
y_pred=clf.predict(X_test)
# Print accuracy score
print("Accuracy: ", accuracy_score(Y_test, y_pred))

Accuracy:  0.8637333333333334


In [24]:
print(classification_report(Y_test, y_pred))

              precision    recall  f1-score   support

          -1       0.87      0.84      0.85      8718
           0       0.84      0.94      0.88      8772
           1       0.90      0.82      0.85      8760

    accuracy                           0.86     26250
   macro avg       0.87      0.86      0.86     26250
weighted avg       0.87      0.86      0.86     26250



In [26]:
# Training a Linear SVC
clf = LinearSVC(random_state=0)
clf.fit(X_train,Y_train)

LinearSVC(random_state=0)

In [27]:
# Input your own tweet
tweet = "I love apples."

# Pre-process the tweet
tweet = BeautifulSoup(tweet).get_text()

# Vectorize the tweet using Tf-idf
tweet_vector = vectorizer.transform([tweet])

# Predict the sentiment of the tweet
sentiment = clf.predict(tweet_vector)

# Print the sentiment
if sentiment == -1:
    print("Negative")
elif sentiment == 0:
    print("Neutral")
else:
    print("Positive")

Positive


In [ ]:
from joblib import dump, load

model_file = "linearSVC.joblib"

dump(clf, model_file)